In [ ]:
#import packages
import pandas as pd
import sys
import matplotlib.pyplot as plt
from datetime import datetime
import logging
import numpy as np
import copy

#custom classes
from classes import classes as cl
from functions import functions as fc
from functions import routing as rt
from functions import simulated_annealing as sa

plt.rcParams.update({'font.size': 22})
logging.basicConfig(filename="rrp_sa_optimization.log", level=logging.INFO)

Initiation

In [ ]:
fc.print_log("Starting initiation")
#create empty dict
initiation_objects = {'dict_depots': '', 'dict_sites': '', 'dict_plants': '', 'dict_jobs': '', 'dict_tours': '', 'list_days': ''}
init_path = 'output/initial_solution'

objects = fc.initiate(initiation_objects,init_path,'final')

#load objects
list_days = objects['list_days']
dict_depots = objects['dict_depots']
dict_sites = objects['dict_sites']
dict_plants = objects['dict_plants']
dict_jobs = objects['dict_jobs']
dict_tours = objects['dict_tours']

#also create lists for convenience
list_depots = [dict_depots[element] for element in dict_depots]
list_sites = [dict_sites[element] for element in dict_sites]
list_plants = [dict_plants[element] for element in dict_plants]
list_jobs = [dict_jobs[element] for element in dict_jobs]
list_tours  = [dict_tours[element] for element in dict_tours]


fc.print_log("Done with initiation")

In the first stage only one depot ist handled, so moves so its all about dropoff and pickup dates

In [ ]:
#ov tours
for depot in dict_depots:
    total_tasks = 0
    total_distance = 0
    for day in list_days:
        total_tasks += dict_tours[depot][day].total_tasks
        total_distance += dict_tours[depot][day].distance
    print("Depot: {}, tasks: {} - distance: {}".format(depot,total_tasks,total_distance))

print("done here")

In [ ]:
print("start here")
fc.print_log("Start Inital Solution Plot")

#only work with one depot
temp_depot = dict_depots['Neuss']
list_depots = [temp_depot]
dict_depots = {'Neuss': temp_depot}



solution_initial = cl.Solution(temp_depot,dict_tours,list_days)

solution_initial.plot_tasks(50)
solution_initial.plot_task_proportion()
solution_initial.plot_distances(10000,1500)

fc.print_log("Done with  Inital Solution Plot")


After having a look at the inital solution, start optimizing

Stage 1: Try to level out the proportions between dropoff and pickup on eatch job

In [ ]:
fc.print_log("Start with Stage 1a")
#dict_tours_temp = copy.deepcopy(dict_tours)
dict_tours_temp = dict_tours
moves_possible = True
move_count = 0
loop_count = 0
step_count = 0

temperature = sa.Geometric_Schedule(1,0.9,100)
dict_statistic = {0: [solution_initial.total_distance, temperature.temp_initial]}


fc.print_log("Start Moves")
while moves_possible and loop_count < 1000:
    move_made = False
    for depot in dict_depots:
        for day_org in list_days:

            #see if there are too many pickups in one job
            tour_org = dict_tours_temp[depot][day_org]

            if tour_org.total_pickups > tour_org.total_dropoffs:
                if tour_org.worst_edge_pickup != '':
                    move_job = tour_org.worst_edge_pickup

                    #see if there is space for a pickup, and check if it works with the earliest pickup date of the job
                    for day_new in list_days:
                        tour_new = dict_tours_temp[depot][day_new]
                        if tour_new.total_pickups < tour_new.total_dropoffs\
                                and move_job.end > day_new:

                            distance_delta = sa.reassign_pickup(tour_org,tour_new,move_job)

                            #adjust move_bol, statistic and leave for loop
                            move_made = True
                            move_count +=1
                            step_count += 1
                            dict_statistic[step_count] = [dict_statistic[step_count-1][0]-distance_delta, temperature.get_temp(step_count)]
                            break

                print("Current: Loop: {} - Moves: {}                                  ".format(loop_count,move_count),end='\r')
                if loop_count > 999: break



    loop_count += 1
    moves_possible = move_made

    print("Current: Loop: {} - Moves: {}                                  ".format(loop_count,move_count),end='\r')

if loop_count > 999:
    fc.print_log("Loopcount exit")

fc.print_log("Final: Loop: {} - Moves: {}".format(loop_count,move_count))

fc.print_log("Done with Moves")
solution_stage1a = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic)

solution_stage1a.plot_tasks(50)
solution_stage1a.plot_task_proportion()
solution_stage1a.plot_developement()


fc.print_log("Done with Stage 1a")
steps_1a = step_count

In [ ]:
print("old distance: %s" % solution_initial.total_distance)
print("new distance: %s" % solution_stage1a.total_distance)

In [ ]:
fc.print_log("Start with Stage 1b")

dict_tours_temp = dict_tours

list_days_reversed = list_days.copy()
list_days_reversed.reverse()


In [ ]:
moves_possible = True
move_count = 0
loop_count = 0
step_count = steps_1a

temperature = sa.Geometric_Schedule(1,0.9,100)
dict_statistic = {step_count: [solution_initial.total_distance, temperature.temp_initial]}

step_count_curr = 0
dict_statistic_curr = {0: [solution_stage1a.total_distance, temperature.temp_initial]}



fc.print_log("Start Moves")
while moves_possible and loop_count < 1000:
    move_made = False
    for depot in dict_depots:
        for day_org in list_days_reversed:

            #see if there are too many pickups in one job
            tour_org = dict_tours_temp[depot][day_org]

            if tour_org.total_dropoffs > tour_org.total_pickups:
                if tour_org.worst_edge_dropoff != '':
                    move_job = tour_org.worst_edge_dropoff

                    #see if there is space for a dropoff, and check if it works with the latest dropoff date of the job
                    for day_new in list_days_reversed:
                        tour_new = dict_tours_temp[depot][day_new]
                        if tour_new.total_dropoffs < tour_new.total_pickups\
                                and move_job.start < day_new:


                            #reassign job
                            distance_delta = sa.reassign_dropoff(tour_org,tour_new,move_job)

                            #adjust move_bol, statistic and leave for loop
                            move_made = True
                            move_count +=1
                            step_count += 1
                            step_count_curr += 1

                            last_total_distance = dict_statistic[step_count-1][0]
                            dict_statistic[step_count] = [last_total_distance + distance_delta,\
                                                          temperature.get_temp(step_count_curr)]
                            dict_statistic_curr[step_count_curr] = dict_statistic[step_count]
                            break

                print("Current: Loop: {} - Moves: {}                                  ".format(loop_count,move_count),end='\r')
                if loop_count > 999: break



    loop_count += 1
    moves_possible = move_made

    print("Current: Loop: {} - Moves: {}                                  ".format(loop_count,move_count),end='\r')

if loop_count > 999:
    fc.print_log("Loopcount exit")

fc.print_log("Final: Loop: {} - Moves: {}".format(loop_count,move_count))

fc.print_log("Done with Moves")
#create new solutions
solution_stage1b = cl.Solution(temp_depot,dict_tours_temp,list_days,\
                               dev_curr=dict_statistic_curr,dev_tot_old=dict_statistic)


solution_stage1b.plot_tasks(50)
solution_stage1b.plot_task_proportion()
fc.print_log("Done with Stage 1b")



In [ ]:
solution_stage1b.plot_developement()
solution_stage1b.plot_developement(total=True)